In [1]:
import pandas as pd
import string
import re
import nltk
import time


stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


df = pd.concat([pd.read_csv('data/data_{}.csv'.format(a)) for a in range(10)])
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df.head()


(9656817, 2)
(8961258, 2)


,Tweet,Category
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,0
1,Loves twitter,0
2,omg so bored &amp; my tattoooos are so itchy!!...,0
3,is going to sleep then on a bike ride:],0
4,RT @SmartChickPDX: Was just told that Nike lay...,0


In [2]:
import numpy as np
np.sum(df['Category'])

2365553

In [3]:
df['Tweet'] = df['Tweet'].astype(str)
df['Len'] = df['Tweet'].str.split().apply(len)
df = df[df['Len']>=4]
print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8430264
Russian 2283309


In [4]:
def clean_text(text):
    # @somone to @
    text = ' '.join(['@' if '@' in a else a for a in text.split(' ')])
    # text = ' '.join(['#' if '#' in a else a for a in text.split(' ')])
    # remove all punctuation except @ and #
    text = ''.join([word.lower() for word in text if word not in string.punctuation.replace('@','').replace('#','')])
    # remove numbers
    text = re.sub('[0-9]+', 'number', text)
    tokens = nltk.word_tokenize(text)
    tokens2 = [a[0] if '@' in a[0] or '#' in a[0] else a[1] for a in nltk.pos_tag(text.split())]
    # hastags only for grammer-check, none in words
    tokens = [a.replace('#','') for a in tokens]
    tokens2 = ['#' if '#' in a else a for a in tokens2]
    lst = []
    lst = [ps.stem(word) for word in tokens]
    for i in range(4):
        for a in nltk.ngrams([ps.stem(word) for word in tokens2],i+3):
            lst.append(' '.join(a))
    
    return lst

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
start = time.time()
countVectorizer = CountVectorizer(analyzer=clean_text
                                  , max_df = .9
                                  , min_df = 0.01 
                                 )
countVector = countVectorizer.fit_transform(df['Tweet'])
end = time.time()

print('{} Number of tweets has {} words and took {:.0f} s'.format(countVector.shape[0], countVector.shape[1],end-start))
# print(countVectorizer.get_feature_names())


8430264 Number of tweets has 430 words and took 15971 s


In [6]:
import pickle
pickle.dump( countVectorizer, open( "files/countVectorizer.pickle", "wb" ) )
pickle.dump( countVector, open( "files/countVector.pickle", "wb" ) )

In [7]:
pickle.dump( df, open( "files/df.pickle", "wb" ) )

In [8]:
print(countVectorizer.get_feature_names())

['', '# NN NN', '@', '@ @ @', '@ @ @ @', '@ @ @ @ @', '@ @ NN', '@ JJ NN', '@ NN @', '@ NN IN', '@ NN NN', '@ NN NN NN', '@ NN RB', '@ NN vbd', '@ NN vbp', '@ nn vbp', 'CC JJ NN', 'DT JJ JJ', 'DT JJ NN', 'DT JJ NN IN', 'DT JJ NN NN', 'DT JJ nn', 'DT NN CC', 'DT NN IN', 'DT NN IN DT', 'DT NN IN DT NN', 'DT NN IN NN', 'DT NN JJ', 'DT NN NN', 'DT NN NN IN', 'DT NN NN NN', 'DT NN RB', 'DT NN TO', 'DT NN vbd', 'DT NN vbz', 'DT nn vbp', 'IN DT JJ', 'IN DT JJ NN', 'IN DT JJ NN NN', 'IN DT NN', 'IN DT NN IN', 'IN DT NN NN', 'IN DT nn', 'IN JJ JJ', 'IN JJ NN', 'IN JJ NN NN', 'IN JJ nn', 'IN NN CC', 'IN NN IN', 'IN NN IN NN', 'IN NN NN', 'IN NN NN NN', 'IN NN RB', 'IN NN nn', 'IN nn vbp', 'IN prp vbp', 'IN prp$ JJ', 'IN prp$ NN', 'IN prp$ NN NN', 'JJ IN NN', 'JJ JJ NN', 'JJ JJ NN NN', 'JJ JJ nn', 'JJ NN #', 'JJ NN @', 'JJ NN CC', 'JJ NN IN', 'JJ NN IN DT', 'JJ NN IN DT NN', 'JJ NN IN JJ', 'JJ NN IN NN', 'JJ NN JJ', 'JJ NN JJ NN', 'JJ NN NN', 'JJ NN NN IN', 'JJ NN NN NN', 'JJ NN NN NN NN', 'JJ NN